In [ ]:
from pynq import Overlay
import os
import sys
import numpy as np

HOP_DIR=os.path.abspath("../")
sys.path.insert(0, HOP_DIR)
import hop

OVERLAY_DIR=f'{HOP_DIR}/overlays/a_add/'

# Preparation

#### Load the overlay onto the FPGA

In [ ]:
ol = Overlay(OVERLAY_DIR + "add.bit")

In [ ]:
ol.ip_dict

#### Register the overlay with the HoP context

In [ ]:
context = hop.Context(ol)

# Use

#### Create some python functions

In [ ]:
def py_func1() -> int:
    return 58

def py_func2() -> int:
    return 1000

def py_func3() -> int:
    return 100

In [ ]:
context.print_all_objects()

#### Register functions with HoP context

In [ ]:
import importlib as il
if 'context' in locals():
    del context
    il.reload(hop)
    hop.Context.reloadModules()
    ol = Overlay(OVERLAY_DIR + "a_add.bit")
    context = hop.Context(ol)

a_py = context.register(py_func1, "b32")
b_py = context.register(py_func2, "b32")
c_py = context.register(py_func3, "b32")
d_constant = context.register(1000, 'b32')

In [ ]:
# Get our hardware
add = context.functions['hardware']['add']

### Use add interchangeably

In [ ]:
print(add(1, np.uint(1000)))
print(add(np.uint32(3), 1))
print(add(np.uint32(50), add(np.uint32(25), np.uint32(25))))
print(add(np.uint32(100), np.uint32(100)))
add(np.uint32(150), np.uint32(150))

### Reduce with hardware

In [ ]:
import functools 
import time

l = [1] * 100
hw_start = time.perf_counter()
print(functools.reduce(add, l))
hw_end = time.perf_counter()
print(f'hw ttf: {hw_end - hw_start}')
sw_start = time.perf_counter()
print(functools.reduce(lambda a,b: a + b, l))
sw_end = time.perf_counter()
print(f'sw ttf: {sw_end - sw_start:.2f}')

#### Imitate Keras Sequential Executor
Tensorflow layers can be hardware, example of defining CNN:

##### Imitate above NN

In [ ]:
import random
class fakeNN:
    def __init__(self, layers):
        self.layers = layers
    
    def singleEpoch(self, inputLayer):
        output = inputLayer
        for l in self.layers:
            leftPercent = random.random()
            rightPercent = 1.0 - leftPercent
            left = int(leftPercent * output)
            right = int(rightPercent * output)
            output = l(left, right)
        return output            

In [ ]:
layers = [add,
          add,
          add]
nn = fakeNN(layers)
print(nn.singleEpoch(32))

# Debug

In [ ]:
add.printRegspacePretty()